In [7]:
import os
import json
import cv2
import numpy as np
import csv
from tqdm import tqdm

# 인스턴스 right / bottom 좌표 -> 높이 / 너비

anno_path = "/data/noah/dataset/ad_human/anno"
out_csv_path = "/data/noah/dataset/magna_traffic_light/height_info.csv"
samples = []

for annotation_name in tqdm(os.listdir(anno_path)):
    with open(os.path.join(anno_path, annotation_name), "r") as f:
        annotation = json.load(f)

    height, width = annotation["metadata"]["height"], annotation["metadata"]["width"]

    for ann in annotation["annotations"]:
        if (
            ann["label"] == "pedestrian"
            and ann["attributes"]["occlusion"] == "0"
            and ann["attributes"]["truncation"] == "0"
        ):
            mask = np.zeros((height, width))

            point = np.array(ann["points"], dtype=np.int32)

            try:
                mask = cv2.fillPoly(mask, [point], color=255).astype(np.uint8)
            except:
                continue

            spots = np.argwhere(mask == 255)
            left = np.min(spots[:, 1])
            right = np.max(spots[:, 1])
            top = np.min(spots[:, 0])
            bottom = np.max(spots[:, 0])

            sample = {"right": right, "bottom": bottom, "height": bottom - top + 100, "width": right - left + 100}
            samples.append(sample)

with open(out_csv_path, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["right", "bottom", "height", "width"])

    writer.writeheader()

    for row in samples:
        writer.writerow(row)

print("CSV 파일이 생성되었습니다.")

100%|██████████| 2632/2632 [00:40<00:00, 64.86it/s]

CSV 파일이 생성되었습니다.


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import csv
import torch


# 데이터셋 생성
# 예제 데이터셋을 생성합니다. 입력은 특정 좌표 (x, y)이고, 출력은 해당 좌표에서의 값입니다.
def generate_dataset(num_samples):
    # 임의의 좌표 생성
    X = np.random.rand(num_samples, 2)
    # 임의의 값 생성
    y = np.random.rand(num_samples, 1)
    return X, y


# 모델 정의
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(2, 16)  # 입력 크기: 2, 은닉층 크기: 64
        self.fc2 = nn.Linear(16, 16)  # 은닉층 크기: 64, 출력 크기: 1
        self.fc3 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# 모델 학습
def train_model(model, samples, num_epochs, learning_rate, device):
    criterion = nn.MSELoss()  # 평균 제곱 오차 손실 함수
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam 옵티마이저

    for epoch in range(num_epochs):
        for sample in samples:
            right, bottom, height, width = sample

            inputs = torch.Tensor([int(right), int(bottom)]).to(device)
            targets = torch.Tensor([int(height)]).to(device)

            # 순전파 및 손실 계산
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # 역전파 및 가중치 업데이트
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


samples = []
out_csv_path = "/data/noah/dataset/magna_traffic_light/height_info.csv"
# CSV 파일을 읽기 모드로 엽니다.
with open(out_csv_path, "r", newline="") as csvfile:
    # CSV 파일을 읽기 위한 reader 객체를 생성합니다.
    csv_reader = csv.reader(csvfile)

    # 각 행을 반복하면서 데이터를 처리합니다.
    for idx, row in enumerate(csv_reader):
        # 각 행의 데이터는 리스트 형태로 반환됩니다.
        # 여기에서 데이터를 사용하거나 출력합니다.
        if not idx:
            continue

        samples.append(row)

device = "cuda:2"
torch.cuda.set_device(device)

# 모델 초기화
model = MLP().to(device)

# 모델 학습
num_epochs = 30
learning_rate = 0.01
train_model(model, samples, num_epochs, learning_rate, device)

ckpt_path = "/data/noah/dataset/magna_traffic_light/height_model.ckpt"
torch.save(model.state_dict(), ckpt_path)

Epoch [5/30], Loss: 2932.4531
Epoch [10/30], Loss: 3225.9612
Epoch [15/30], Loss: 3172.4348
Epoch [20/30], Loss: 3201.7507
Epoch [25/30], Loss: 3246.0156
Epoch [30/30], Loss: 3258.4976


In [ ]:
model.load_state_dict(torch.load(ckpt_path))

# 테스트 데이터 입력
# 32, 15
input_coordinate = [1312, 566]

# 테스트 데이터 예측
input_tensor = torch.Tensor(input_coordinate).to(device)
predicted_value = model(input_tensor)
print(predicted_value.detach().cpu().numpy())